# Yelp API - Lab

The previously deployed lab on working around building a GIS with Yelp API and Folium can be found [here](https://github.com/learn-co-curriculum/dsc-2-15-10-yelp-api-gis-lab/tree/a56358c2d0c2daf569a5f50937c4c27463aadb1a) (not relevant for new students).


## Introduction 

Now that we've seen how the Yelp API works, and some basic Folium visualizations its time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and perform data analysis on the data returned
* Perform pagination to retrieve troves of data!
* Create a simple geographical system on to view information about selected businesses, at a given location. 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you, but make the request below.

In [1]:
#Your code here
import time
import requests 

client_id = '5onuGByRauHpN8oN0PEgtw' #Your client ID goes here (as a string)
api_key = 'Ystbu1NrD08vSyMfFl5XxExXnikbKXwv-MtJRUHCSQNjTYWTSpsn7SRkLQUoVE9nf1nr_JCD7oD9kCjQIgMGQbxKFF-HwpB0dresyDzXqx-nztbSq4rhrVq0MHJcXHYx'


def get_all_info():    
    data = []
    term = 'Japanese'
    location = 'Flatiron School'
    SEARCH_LIMIT = 50
    price = "2"
    sort_by = "rating"
    radius = 500
    offset = 0
    search_count = SEARCH_LIMIT

    url = 'https://api.yelp.com/v3/businesses/search'

    headers = {
            'Authorization': 'Bearer {}'.format(api_key),
        }

    while search_count >= 0:
        url_params = {
                        'term': term.replace(' ', '+'),
                        'location': location.replace(' ', '+'),
                        'price':price,
                        'sort_by':sort_by,
                        'limit': SEARCH_LIMIT,
                        'radius':radius,
                        'offset':offset
                    }
        yelp_api = requests.get(url, headers=headers, params=url_params)
        data.append(yelp_api.json())
        offset += SEARCH_LIMIT
        search_count = data[0]["total"] - offset
        
    return data
data = get_all_info()
data

[{'businesses': [{'id': 'k2p78r6NOhYYb_5QOORGcA',
    'alias': 'kome-waza-new-york',
    'name': 'Kome Waza',
    'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/Z5e71qnRHbhdLYo8Xl74HA/o.jpg',
    'is_closed': False,
    'url': 'https://www.yelp.com/biz/kome-waza-new-york?adjust_creative=5onuGByRauHpN8oN0PEgtw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=5onuGByRauHpN8oN0PEgtw',
    'review_count': 153,
    'categories': [{'alias': 'japanese', 'title': 'Japanese'},
     {'alias': 'salad', 'title': 'Salad'},
     {'alias': 'poke', 'title': 'Poke'}],
    'rating': 4.0,
    'coordinates': {'latitude': 40.70351, 'longitude': -74.01024},
    'transactions': ['pickup', 'delivery'],
    'price': '$$',
    'location': {'address1': '40 Water St',
     'address2': '',
     'address3': '',
     'city': 'New York',
     'zip_code': '10004',
     'country': 'US',
     'state': 'NY',
     'display_address': ['40 Water St', 'New York, NY 10004']},
    'phone': '+1646368

## Pagination

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.josn().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [ ]:
# Your code here; use a function or loop to retrieve all the results from your original request

## Exploratory Analysis

Take the restaurants from the previous question and do an intial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [4]:
#Your code here
import pandas as pd
import numpy as np
parsed_data = []

for information in data:
    for business in information["businesses"]:
        parsed_dict = {}
        parsed_dict["business_name"] = business["name"]
        parsed_dict["address"] = (" ").join(business["location"]["display_address"])
        parsed_dict["phone"] = business["phone"]
        parsed_dict["transactions"] = business["transactions"]
        parsed_dict["rating"] = business["rating"]
        parsed_dict["review_count"] = business["review_count"]
        parsed_data.append(parsed_dict)
yelp_df = pd.DataFrame(data=parsed_data)
yelp_df

,address,business_name,phone,rating,review_count,transactions
0,"40 Water St New York, NY 10004",Kome Waza,+16463681727,4.0,152,"[delivery, pickup]"
1,"80 Broad St New York, NY 10004",Pokebab,+16463516011,4.5,36,"[delivery, pickup]"
2,"3 Hanover Sq New York, NY 10004",Poke Green,+12123613777,4.0,40,"[delivery, pickup]"
3,"60 Beaver St New York, NY 10004",Taste of Tokyo,+12128094711,3.0,140,"[delivery, pickup]"
4,"38 Water St New York, NY 10004",OBAO,+12123616311,3.0,195,"[delivery, pickup]"
5,"94 1/2 Greenwich St New York, NY 10006",Cafe De Novo,+12122331573,3.0,85,"[delivery, pickup]"
6,"11 Stone St New York, NY 10004",Asian Fusion On Stone,+12127421188,2.5,26,"[delivery, pickup]"


## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [11]:
#Your code here
!pip install folium
import matplotlib as plt
import folium

You are using pip version 10.0.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [14]:
folium.map()

TypeError: 'module' object is not callable

## Summary

Nice work! In this lab, you synthesized your skills for the day, making multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice map visual to display the results! Well done!